In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janatahack-computer-vision/data/train.csv
/kaggle/input/janatahack-computer-vision/data/test.csv
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1031.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1024.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1009.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1026.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1010.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1032.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1002.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/994.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1019.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/2332.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/1018.jpg
/kaggle/input/janatahack-computer-vision/data/val/noemergency/2347.jpg
/kaggle/input/janatahack-computer-visi

In [2]:
import cv2 as cv
import pandas as pd 
import numpy as np 

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,AveragePooling2D,BatchNormalization,Dense,Dropout
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16,ResNet50,DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras import models
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import *

import random
random.seed(42)

In [3]:
traindir = "../input/janatahack-computer-vision/data/train"
testdir = "../input/janatahack-computer-vision/data/val"

In [4]:
train_datagen=ImageDataGenerator(horizontal_flip=True , 
                                 preprocessing_function=preprocess_input,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 rotation_range=30
                                )

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [5]:
train_generator=train_datagen.flow_from_directory(
    traindir,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=64
)

test_generator=test_datagen.flow_from_directory(
testdir,
target_size =(224,224),
class_mode="categorical",
batch_size=64
)

Found 1533 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [6]:
print(train_generator.classes)
print(train_generator.class_indices)

[0 0 0 ... 1 1 1]
{'emergency': 0, 'noemergency': 1}


RESNET BUILDING


In [7]:
resnet=DenseNet121(weights='imagenet',include_top=False ,input_shape=(224,224,3))

model=models.Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.summary()

29089792/29084464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 100354    
Total params: 7,137,858
Trainable params: 7,054,210
Non-trainable params: 83,648
_________________________________________________________________


In [8]:
sgd = optimizers.Nadam(lr=0.00001)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [9]:
history=model.fit_generator(train_generator ,
                           epochs=5,
                           validation_data=test_generator,
                           )

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
24/24 [==============================] - 60s 1s/step - loss: 0.9240 - accuracy: 0.5675 - val_loss: 0.6270 - val_accuracy: 0.7200
Epoch 2/5
24/24 [==============================] - 20s 844ms/step - loss: 0.5656 - accuracy: 0.7475 - val_loss: 0.3984 - val_accuracy: 0.8300
Epoch 3/5
24/24 [==============================] - 21s 863ms/step - loss: 0.4412 - accuracy: 0.8040 - val_loss: 0.3103 - val_accuracy: 0.8800
Epoch 4/5
24/24 [==============================] - 20s 839ms/step - loss: 0.3580 - accuracy: 0.8480 - val_loss: 0.2646 - val_accuracy: 0.9300
Epoch 5/5
24/24 [==============================] - 21s 848ms/step - loss: 0.3031 - accuracy: 0.8818 - val_loss: 0.2496 - val_accuracy: 0.9300


In [10]:
model.save("model.h5")

In [11]:
#loading the model
model=load_model("./model.h5")

In [12]:
testdir = "../input/janatahack-computer-vision/data/test"

In [13]:
for i in os.listdir():
    print(i[-4:])

l.h5
pynb


In [14]:
images=[]
names=[]

path='../input/janatahack-computer-vision/data/test'
dir_list = os.listdir(path)

for i in os.listdir():
    if i[-4:]!='.jpg':
        continue
    img=cv.imread(i)
    img=preprocess_input(img)
    images.append(img)
    names.append(i)

In [15]:
os.listdir

<function posix.listdir(path=None)>

In [19]:
testdir = "../input/janatahack-computer-vision/data/test"
os.chdir(testdir)
images=[]
names=[]
for i in os.listdir():
    if i[-4:]!=".jpg":
        continue
    img=cv.imread(i)
    img=preprocess_input(img)
    images.append(img)
    names.append(i)

FileNotFoundError: [Errno 2] No such file or directory: '../input/janatahack-computer-vision/data/test'

In [17]:
os.listdir

<function posix.listdir(path=None)>

In [20]:
data=np.array(images)
data.shape

(706, 224, 224, 3)

In [21]:
preds=model.predict(data)

In [22]:
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

max_pred = np.argmax(preds, axis=1)
max_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,

In [23]:
final_output = abs(1 - max_pred)

In [24]:
result = pd.DataFrame({"image_names":names,"emergency_or_not": final_output})
result.head(10)

result.to_csv('/kaggle/working/submission.csv' , index = None)